In [1]:
import pandas as pd
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
import math

import numpy as np

In [2]:
path = '/home/simone/Desktop/NLFF/intrinioDatasetUpdated/AAPL'
tickFiles = [f for f in listdir('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/') if isfile(join('/home/simone/Desktop/NLFF/intrinioDatasetUpdated/', f))]

news = pd.read_csv(path+'.csv')
count =1
while 'AAPL'+str(count)+'.csv' in tickFiles:
    print(path+str(count))
    newsTemp = pd.read_csv(path +str(count)+'.csv')
    news = pd.concat([news, newsTemp])
    count+=1
news.drop_duplicates(subset=['PUBLICATION_DATE'], inplace=True)
news = news.sort_values(by=['PUBLICATION_DATE'])
news = news.reset_index(drop=True)
    
aapl = news
for i in range(len(aapl)):
    aapl.at[i,'PUBLICATION_DATE'] =datetime.strptime(aapl['PUBLICATION_DATE'][i], '%Y-%m-%d %H:%M:%S +%f') 
aapl.head()

/home/simone/Desktop/NLFF/intrinioDatasetUpdated/AAPL1
/home/simone/Desktop/NLFF/intrinioDatasetUpdated/AAPL2


,TICKER,FIGI_TICKER,FIGI,TITLE,PUBLICATION_DATE,URL,SUMMARY
0,AAPL,AAPL:UW,BBG000B9Y5X2,The tiniest tweak to Apple’s supply chain can ...,2017-04-03 17:27:34,https://qz.com/948375/apples-aapl-latest-tweak...,The tiniest tweak to Apple’s supply chain can ...
1,AAPL,AAPL:UW,BBG000B9Y5X2,Apple Roundup: iPhone Still Drives Apple Shares,2017-04-03 17:32:05,http://finance.yahoo.com/news/apple-roundup-ip...,Apple Roundup: iPhone Still Drives Apple Shares
2,AAPL,AAPL:UW,BBG000B9Y5X2,1 More Negative for This Small Apple Inc. Supp...,2017-04-03 17:36:40,http://articlefeeds.nasdaq.com/~r/nasdaq/symbo...,By this point the tech investment community is...
3,AAPL,AAPL:UW,BBG000B9Y5X2,"Apple releases iOS 10.3.1 for iPhone, iPad",2017-04-03 17:40:00,http://www.zdnet.com/article/apple-releases-io...,"Apple releases iOS 10.3.1 for iPhone, iPad"
4,AAPL,AAPL:UW,BBG000B9Y5X2,Imagination Technologies' shares plunge 70 pct...,2017-04-03 17:42:47,http://uk.finance.yahoo.com/news/imagination-t...,[Reuters - UK Focus] - Imagination Technologie...


In [3]:
#aapl = aapl[aapl['PUBLICATION_DATE'] > ]
aapl.drop(aapl[aapl.PUBLICATION_DATE < datetime(2018, 1, 1, 0, 0, 0)].index, inplace=True)
aapl.drop(aapl[aapl.PUBLICATION_DATE > datetime(2018, 4, 1, 0, 0, 0)].index, inplace=True)

aapl.head()

,TICKER,FIGI_TICKER,FIGI,TITLE,PUBLICATION_DATE,URL,SUMMARY
18028,AAPL,AAPL:UW,BBG000B9Y5X2,Bitcoin or Stocks? Here’s the One to Buy in 2018,2018-01-01 16:06:40,http://articlefeeds.nasdaq.com/~r/nasdaq/symbo...,By Michael Foster Worried that a bursting bitc...
18029,AAPL,AAPL:UW,BBG000B9Y5X2,2 Warren Buffett Stocks to Consider Buying Now,2018-01-01 20:02:00,https://finance.yahoo.com/news/2-warren-buffet...,The legendary investor knows a thing or two ab...
18030,AAPL,AAPL:UW,BBG000B9Y5X2,2 Warren Buffett Stocks to Consider Buying Now,2018-01-01 21:08:20,http://articlefeeds.nasdaq.com/~r/nasdaq/symbo...,A 1 000 investment in Berkshire Hathaway 160 s...
18031,AAPL,AAPL:UW,BBG000B9Y5X2,Apple's Executive Cash Bonus Plan,2018-01-01 22:19:00,http://finance.yahoo.com/r/a212bfe1-4a00-3d00-...,Apple’s executives have done very well over th...
18032,AAPL,AAPL:UW,BBG000B9Y5X2,"Forget Your iPhone X, Ignore The Samsung And P...",2018-01-01 23:28:00,http://finance.yahoo.com/r/2e9f187e-56ab-319e-...,"Looking back on 2017, the smartphone industry ..."


In [4]:
len(aapl)

4979

In [7]:
count = 0
for index, row in aapl.iterrows():
    count += math.ceil(len(row['SUMMARY'])/1000)
    
print(count)

4984


In [2]:
asd = pd.read_csv('/home/simone/Desktop/NLFF/GoogleNL/googleSentiment.csv', sep=';')
asd.head()

,DATE,GoogleSentiment
0,2018-01-01 16:06:40,"""entities { name: ""Michael Foster"" type: PE..."
1,2018-01-01 20:02:00,"""entities { name: ""investor"" type: PERSON ..."
2,2018-01-01 21:08:20,"""entities { name: ""investment"" type: OTHER ..."
3,2018-01-01 22:19:00,"""entities { name: ""executives"" type: PERSON..."
4,2018-01-01 23:28:00,"""entities { name: ""settle"" type: OTHER sal..."


In [3]:
len(asd)

1623

In [6]:
import argparse
import sys
import pandas as pd
import numpy as np
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types
import six
from datetime import datetime, timedelta
from os import listdir
from os.path import isfile, join
import math
import json


# Instantiates a client
client = language.LanguageServiceClient()

In [2]:

def entity_sentiment_text(text):
    """Detects entity sentiment in the provided text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    document = types.Document(
        content=text.encode('utf-8'),
        type=enums.Document.Type.PLAIN_TEXT)

    # Detect and send native Python encoding to receive correct word offsets.
    encoding = enums.EncodingType.UTF32
    if sys.maxunicode == 65535:
        encoding = enums.EncodingType.UTF16

    result = client.analyze_entity_sentiment(document, encoding)

    return result

In [3]:
gResult = entity_sentiment_text('A new program for hotels and rentals lets guests use Alexa for services like music, lights, and temperature.')

In [4]:
type(gResult)

google.cloud.language_v1.types.AnalyzeEntitySentimentResponse

In [11]:
gResult

entities {
  name: "program"
  type: OTHER
  salience: 0.3639371991157532
  mentions {
    text {
      content: "program"
      begin_offset: 6
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "hotels"
  type: LOCATION
  salience: 0.1463717669248581
  mentions {
    text {
      content: "hotels"
      begin_offset: 18
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "rentals"
  type: OTHER
  salience: 0.10108248889446259
  mentions {
    text {
      content: "rentals"
      begin_offset: 29
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "services"
  type: OTHER
  salience: 0.09421800076961517
  mentions {
    text {
      content: "services"
      begin_offset: 63
    }
    type: COMMON
    sentiment {
    }
  }
  sentiment {
  }
}
entities {
  name: "guests"
  type: PERSON
  salience: 0.07801704853773117
  mentions {
    text {
      content: "guests"
      begin_o

In [37]:
data = {}
entities = list()
data['entities'] = entities
for entity in gResult.entities:
    entityJ = {}
    entities.append(entityJ)
    mentionsJ = list()

    entityJ['name'] = entity.name if entity.name is not None else ''
    entityJ['salience'] = entity.salience if entity.salience is not None else ''
    entityJ['sentiment'] = {'magnitude': entity.sentiment.magnitude, 'score':entity.sentiment.score}
    entityJ['type'] = entity.type if entity.type is not None else ''
    entityJ['mentions'] = mentionsJ    
    

    for mention in entity.mentions:
        sentiment = {}
        text = {}
        text['content'] = mention.text.content
        text['begin_offset'] = mention.text.begin_offset
        sentiment['magnitude'] = mention.sentiment.magnitude
        sentiment['score'] = mention.sentiment.score
        mentionsJ.append({'text':text, 'type':mention.type, 'sentiment':sentiment})

data['entities'] = entities
data['language'] = gResult.language
json_data = json.dumps(data)

In [35]:
json_data

'{"entities": [{"name": "program", "salience": 0.3639371991157532, "sentiment": {"magnitude": 0.0, "score": 0.0}, "type": 7, "mentions": [{"text": {"content": "program", "begin_offset": 6}, "type": 2, "sentiment": {"magnitude": 0.0, "score": 0.0}}]}, {"name": "hotels", "salience": 0.1463717669248581, "sentiment": {"magnitude": 0.0, "score": 0.0}, "type": 2, "mentions": [{"text": {"content": "hotels", "begin_offset": 18}, "type": 2, "sentiment": {"magnitude": 0.0, "score": 0.0}}]}, {"name": "rentals", "salience": 0.10108248889446259, "sentiment": {"magnitude": 0.0, "score": 0.0}, "type": 7, "mentions": [{"text": {"content": "rentals", "begin_offset": 29}, "type": 2, "sentiment": {"magnitude": 0.0, "score": 0.0}}]}, {"name": "services", "salience": 0.09421800076961517, "sentiment": {"magnitude": 0.0, "score": 0.0}, "type": 7, "mentions": [{"text": {"content": "services", "begin_offset": 63}, "type": 2, "sentiment": {"magnitude": 0.0, "score": 0.0}}]}, {"name": "guests", "salience": 0.078